# Introduction to the Keras Tuner

## Overview

The Keras Tuner is a library that helps you pick the optimal set of hyperparameters for your TensorFlow program.

Hyperparameters are of two types:
1. **Model hyperparameters** which influence model selection such as the number and width of hidden layers
2. **Algorithm hyperparameters** which influence the speed and quality of the learning algorithm such as the learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier

## Setup

In [1]:
import tensorflow as tf
from tensorflow import keras

Install and import the Keras Tuner.

In [2]:
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.3 MB/s eta 0:00:00


In [3]:
import keras_tuner as kt

## Download and prepare the dataset

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [5]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Define the model

A hypermodel cab be defined through two approaches:

* By using a model builder function
* By subclassing the `HyperModel` class of the Keras Tuner API

A model builder function is used to define the image classification model. The model builder function returns a compiled model and uses hyperparameters you define inline to hypertune the model.

In [6]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

## Instantiate the tuner and perform hypertuning

Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - `RandomSearch`, `Hyperband`, `BayesianOptimization`, and `Sklearn`.

To instantiate the Hyperband tuner, the hypermodel, the `objective` to optimize and the maximum number of epochs to train (`max_epochs`) should be specified.

In [7]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. Hyperband determines the number of models to train in a bracket by computing 1 + log<sub>`factor`</sub>(`max_epochs`) and rounding it up to the nearest integer.

Create a callback to stop training early after reaching a certain value for the validation loss.

In [8]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Run the hyperparameter search. The arguments for the search method are the same as those used for `tf.keras.model.fit` in addition to the callback above.

In [9]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 48s]
val_accuracy: 0.8608333468437195

Best val_accuracy So Far: 0.8914999961853027
Total elapsed time: 00h 14m 01s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 256 and the optimal learning rate for the optimizer
is 0.001.



## Train the model

Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.

In [10]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5087 - accuracy: 0.8200 - val_loss: 0.4071 - val_accuracy: 0.8534
Epoch 2/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3738 - accuracy: 0.8651 - val_loss: 0.3577 - val_accuracy: 0.8716
Epoch 3/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3331 - accuracy: 0.8773 - val_loss: 0.3507 - val_accuracy: 0.8758
Epoch 4/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3086 - accuracy: 0.8862 - val_loss: 0.3249 - val_accuracy: 0.8811
Epoch 5/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2893 - accuracy: 0.8941 - val_loss: 0.3094 - val_accuracy: 0.8882
Epoch 6/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2752 - accuracy: 0.8979 - val_loss: 0.3271 - val_accuracy: 0.8832
Epoch 7/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2615 - accuracy: 0.9029 - val_loss: 0.3148 - val_accuracy:

Re-instantiate the hypermodel and train it with the optimal number of epochs from above.

In [11]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/33
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5023 - accuracy: 0.8240 - val_loss: 0.3861 - val_accuracy: 0.8627
Epoch 2/33
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3751 - accuracy: 0.8617 - val_loss: 0.3701 - val_accuracy: 0.8664
Epoch 3/33
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3354 - accuracy: 0.8779 - val_loss: 0.3685 - val_accuracy: 0.8635
Epoch 4/33
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3086 - accuracy: 0.8864 - val_loss: 0.3419 - val_accuracy: 0.8785
Epoch 5/33
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2928 - accuracy: 0.8912 - val_loss: 0.3540 - val_accuracy: 0.8748
Epoch 6/33
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2755 - accuracy: 0.8980 - val_loss: 0.3391 - val_accuracy: 0.8783
Epoch 7/33
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2610 - accuracy: 0.9044 - val_loss: 0.3411 - val_accuracy:

Evaluate the hypermodel on the test data.

In [12]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 3ms/step - loss: 0.4767 - accuracy: 0.8859
[test loss, test accuracy]: [0.4767216444015503, 0.8859000205993652]


The `my_dir/intro_to_kt` directory contains detailed logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional `overwrite=True` argument while instantiating the tuner.